# Data Prep

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sys, os
sys.path.append(os.path.abspath(".."))

In [2]:
from src.preprocess.preprocessor import get_df
from src.config_training import TrainSettings

settings = TrainSettings()

TRAIN_START_DATE = settings.TRAIN_START_DATE
TRAIN_END_DATE = settings.TRAIN_END_DATE
VAL_START_DATE = settings.VAL_START_DATE
VAL_END_DATE = settings.VAL_END_DATE

TEST23_START_DATE = settings.TEST23_START_DATE
TEST23_END_DATE = settings.TEST23_END_DATE
TEST24_START_DATE = settings.TEST24_START_DATE
TEST24_END_DATE = settings.TEST24_END_DATE

INDICATORS = settings.INDICATORS

df_train = get_df(TRAIN_START_DATE, TRAIN_END_DATE, "../src/preprocess/tickers/ticker_lists.json")
df_val = get_df(VAL_START_DATE, VAL_END_DATE, "../src/preprocess/tickers/ticker_lists.json")
df_test23 = get_df(TEST23_START_DATE, TEST23_END_DATE, "../src/preprocess/tickers/ticker_lists.json")
df_test24 = get_df(TEST24_START_DATE, TEST24_END_DATE, "../src/preprocess/tickers/ticker_lists.json")
stock_dimension = len(df_train.tic.unique())


From: 2017-12-02 00:00:00
To: 2022-01-01
Shape of DataFrame:  (30810, 8)
Successfully added technical indicators
From: 2021-12-02 00:00:00
To: 2023-01-01
Shape of DataFrame:  (8160, 8)
Successfully added technical indicators
From: 2022-12-02 00:00:00
To: 2024-01-01
Shape of DataFrame:  (8100, 8)
Successfully added technical indicators
From: 2023-12-02 00:00:00
To: 2025-01-01
Shape of DataFrame:  (8130, 8)
Successfully added technical indicators


# metaHRL

## Training

In [3]:
# TRAINING with BEST HPs
from env_stocktrading.trading_env_HRL import StockTradingEnvHRL
from agent.metaHRL_reptile import MetaHRLAgent
from agent.meta.task_config import MetaTrainHelper

episode_len = df_train.dayorder.nunique()
state_space_manager = stock_dimension + len(INDICATORS) * stock_dimension
state_space_worker = 1 + 3 * stock_dimension

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

hrl_train_env = StockTradingEnvHRL(
    df=df_train,
    stock_dim=stock_dimension,
    hmax=100,
    initial_amount=1000000,
    num_stock_shares=num_stock_shares,
    buy_cost_pct=buy_cost_list,
    sell_cost_pct=sell_cost_list,
    state_space_M=state_space_manager,
    state_space_W=state_space_worker,
    action_space=stock_dimension,
    tech_indicator_list=INDICATORS,
    make_plots=False,
    print_verbosity=1,
)

best_params = settings.best_hiperparams_HRL_worker_update

best_params_manager = {
    "lr_actor": best_params["lr_actor_M"],
    "lr_critic": best_params["lr_critic_M"],
    "gamma": best_params["gamma_M"],
    "K_epochs": 80,
    "eps_clip": 0.2,
    "update_timestep": best_params["update_timestep"],
}

best_params_worker = {
    "learning_rate": best_params["lr_W"],
    "tau": 0.005,
    "buffer_size": best_params["buffer_size"],
    "batch_size": best_params["batch_size"],
    "gamma": best_params["gamma_W"],
    "verbose": 1,
}

initial_manager_episodes = 25
initial_worker_episodes = 30#35
initial_cycle_episodes = 10#10
n_cycles = 12#16

task_helper = MetaTrainHelper(json_path="../src/preprocess/tickers/ticker_lists.json")

model_meta = MetaHRLAgent(
    task_helper=task_helper,
    env=hrl_train_env,
    stock_dim=stock_dimension,
    manager_kwargs=best_params_manager,
    worker_kwargs=best_params_worker,
    initial_manager_timesteps=initial_manager_episodes * episode_len,  # initial_manager_episodes
    initial_worker_timesteps=initial_worker_episodes * episode_len,   # initial_worker_episodes
    n_alt_cycles=n_cycles,                                              # n_cycles
    initial_cycle_steps=initial_cycle_episodes * episode_len,        # initial_cycle_episodes
)

trained_model_meta = model_meta.train_HRL_model()

Device set to : cpu
From: 2021-12-02 00:00:00
To: 2023-01-01
Shape of DataFrame:  (8160, 8)
Successfully added technical indicators
From: 2017-12-02 00:00:00
To: 2020-01-01
Shape of DataFrame:  (15660, 8)
Successfully added technical indicators
From: 2014-12-02 00:00:00
To: 2017-01-01
Shape of DataFrame:  (15750, 8)
Successfully added technical indicators
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

=== Phase 1: Meta-Training Manager ===
--- [Meta] Iniciando Reptile (Meta-LR: 0.001) ---
Task MarketTask.STAGNANT, Meta Epoch: 0
day: 503, episode: 1
begin_total_asset: 1000000.00
end_total_asset: 1098813.85
total_reward: 98813.85
total_cost: 28957.27
total_trades: 9392
Sharpe: 0.401
Task MarketTask.BEARISH, Meta Epoch: 0
day: 250, episode: 1
begin_total_asset: 1000000.00
end_total_asset: 919414.25
total_reward: -80585.75
total_cost: 29937.13
total_trades: 4451
Sharpe: -0.314
day: 250, episode: 2
begin_total_asset: 1000000.00
end_total_asset

In [4]:
# VALIDATION
hrl_val_env = StockTradingEnvHRL(
    df=df_val,
    stock_dim=stock_dimension,
    hmax=100,
    initial_amount=1000000,
    num_stock_shares=num_stock_shares,
    buy_cost_pct=buy_cost_list,
    sell_cost_pct=sell_cost_list,
    state_space_M=state_space_manager,
    state_space_W=state_space_worker,
    action_space=stock_dimension,
    tech_indicator_list=INDICATORS,
    make_plots=False,
    print_verbosity=1,
)

df_account_value_hrlmeta, df_actions_hrlmeta, last_state_hrlmeta = trained_model_meta.predictHRL(hrl_val_env)

Retrieving memory from env...
day: 250, episode: 1
begin_total_asset: 1000000.00
end_total_asset: 835751.36
total_reward: -164248.64
total_cost: 5078.13
total_trades: 1353
Sharpe: -0.530
Retrieving memory from env...
Hit end!


In [8]:
# AÑO 2023
test23_env = StockTradingEnvHRL(
    df = df_test23,
    stock_dim=stock_dimension,
    hmax=100,
    initial_amount=1000000,
    num_stock_shares=num_stock_shares,
    buy_cost_pct=buy_cost_list,
    sell_cost_pct=sell_cost_list,
    state_space_M=state_space_manager,
    state_space_W=state_space_worker,
    action_space=stock_dimension,
    tech_indicator_list=INDICATORS,
    make_plots=False,
    print_verbosity=1,
)

df_account_test23_HRLmeta, df_actions_test23_HRLmeta, last_state_test23_HRLmeta = trained_model_meta.predictHRL(test23_env)

#df_account_test23_HRLmeta.to_csv("../demo/results/dfs/cuenta_test23_metaHRL.csv")
print(last_state_test23_HRLmeta[31:61])

Retrieving memory from env...
day: 249, episode: 1
begin_total_asset: 1000000.00
end_total_asset: 1162715.56
total_reward: 162715.56
total_cost: 4755.23
total_trades: 1381
Sharpe: 1.127
Retrieving memory from env...
Hit end!
[344.0, 1188.0, 0, 1940.0, 2.0, 0.0, 201.0, 0.0, 82.0, 0.0, 0.0, 0.0, 119.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 116.0, 0, 0.0, 0.0, 0, 430.0, 0.0, 0.0, 2018.0]


In [6]:
# AÑO 2024
test24_env = StockTradingEnvHRL(
    df = df_test24,
    stock_dim=stock_dimension,
    hmax=100,
    initial_amount=1000000,
    num_stock_shares=num_stock_shares,
    buy_cost_pct=buy_cost_list,
    sell_cost_pct=sell_cost_list,
    state_space_M=state_space_manager,
    state_space_W=state_space_worker,
    action_space=stock_dimension,
    tech_indicator_list=INDICATORS,
    make_plots=False,
    print_verbosity=1,
)

df_account_test24_HRL, df_actions_test24_HRL, last_state_test24_HRL = trained_model_meta.predictHRL(test24_env)

df_account_test24_HRL.to_csv("../demo/results/dfs/cuenta_test24_metaHRL.csv")
print(last_state_test24_HRL[31:61])

Retrieving memory from env...
day: 251, episode: 1
begin_total_asset: 1000000.00
end_total_asset: 1413592.09
total_reward: 413592.09
total_cost: 4951.96
total_trades: 1271
Sharpe: 2.199
Retrieving memory from env...
Hit end!
[835.0, 54.0, 0, 2205.0, 108.0, 0, 0.0, 113.0, 0.0, 300.0, 0.0, 0.0, 0.0, 0.0, 0.0, 600.0, 0.0, 0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0, 485.0, 0.0, 0.0, 1177.0]
